In [1]:
# Cell 0: Set Environment Variables

import os

# Set FINN_BUILD_DIR to the build output directory
os.environ["FINN_BUILD_DIR"] = os.path.abspath("../build")

print(f"✓ FINN_BUILD_DIR set to: {os.environ['FINN_BUILD_DIR']}")

✓ FINN_BUILD_DIR set to: /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/build


In [2]:
# Cell 1: Imports and Setup

import os
import sys
import shutil
import numpy as np
from qonnx.core.modelwrapper import ModelWrapper

# FINN imports
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
from finn.builder.build_dataflow_config import DataflowBuildConfig

print("✓ Imports successful")
print(f"  Python: {sys.version}")
print(f"  Working directory: {os.getcwd()}")

# Test FINN
try:
    from finn.util.basic import get_finn_root
    print(f"✓ FINN root: {get_finn_root()}")
except:
    print("✓ FINN imported successfully")

W1218 15:16:12.825714 6368 site-packages/torch/utils/cpp_extension.py:118] No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


✓ Imports successful
  Python: 3.9.25 (main, Nov  3 2025, 22:33:05) 
[GCC 11.2.0]
  Working directory: /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/notebooks
✓ FINN imported successfully


In [3]:
# Cell 2: Verify Model Exists

# Path to your streamlined QONNX model
model_path = "../../finn_build/ellipse_qonnx_streamlined.qonnx"

# Check if model exists
if not os.path.exists(model_path):
    raise FileNotFoundError(
        f"Model not found: {model_path}\n"
        "Please run 2-finn.ipynb first to generate the streamlined model."
    )

# Load and inspect the model
model = ModelWrapper(model_path)
print(f"✓ Model loaded: {model_path}")
print(f"  Input: {model.graph.input[0].name} {[d.dim_value for d in model.graph.input[0].type.tensor_type.shape.dim]}")
print(f"  Output: {model.graph.output[0].name} {[d.dim_value for d in model.graph.output[0].type.tensor_type.shape.dim]}")
print(f"  Total nodes: {len(model.graph.node)}")

# Count node types
from collections import Counter
node_types = Counter([n.op_type for n in model.graph.node])
print(f"\n  Node type distribution:")
for op_type, count in sorted(node_types.items(), key=lambda x: -x[1])[:10]:
    print(f"    {op_type}: {count}")

✓ Model loaded: ../../finn_build/ellipse_qonnx_streamlined.qonnx
  Input: global_in [0, 1, 20, 20]
  Output: global_out [1, 5]
  Total nodes: 63

  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Conv: 4
    MaxPool: 4
    Add: 3
    Gemm: 3


In [5]:
# Cell 3: Configure Build Settings

# Output directory for build artifacts
output_dir = "../build"

# Clean output directory if it exists
if os.path.exists(output_dir):
    print(f"  Cleaning existing build directory: {output_dir}")
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

print(f"✓ Build configuration:")
print(f"  Model: {model_path}")
print(f"  Output: {output_dir}")
print(f"  Target board: Kria KV260")

  Cleaning existing build directory: ../build
✓ Build configuration:
  Model: ../../finn_build/ellipse_qonnx_streamlined.qonnx
  Output: ../build
  Target board: Kria KV260


In [6]:
# Cell 4: Create Dataflow Build Configuration (Based on FINN Cybersecurity Example)
from finn.builder.build_dataflow_steps import (
    step_qonnx_to_finn,
    step_tidy_up,
    step_streamline,
    step_convert_to_hw,
    # Skip step_create_dataflow_partition - causes cycle errors in some models
    step_target_fps_parallelization,
    step_apply_folding_config,
    step_minimize_bit_width,
    step_generate_estimate_reports,
    step_hw_codegen,
    step_hw_ipgen,
    step_set_fifo_depths,
    step_create_stitched_ip,
    step_out_of_context_synthesis,
    step_synthesize_bitfile,
    step_make_pynq_driver,
    step_deployment_package,
)

# Build steps following FINN cybersecurity example pattern
build_steps = [
    step_qonnx_to_finn,
    step_tidy_up,
    step_streamline,
    step_convert_to_hw,
    # Partitioning skipped - not needed for single-partition designs
    step_target_fps_parallelization,
    step_apply_folding_config,
    step_minimize_bit_width,
    step_generate_estimate_reports,
    step_hw_codegen,
    step_hw_ipgen,
    step_set_fifo_depths,
    step_create_stitched_ip,
    step_out_of_context_synthesis,
    step_synthesize_bitfile,
    step_make_pynq_driver,
    step_deployment_package,
]

# Kria KV260 configuration
cfg = DataflowBuildConfig(
    output_dir=output_dir,
    # Target FPGA part for Kria KV260
    fpga_part="xck26-sfvc784-2LV-c",
    # Clock frequency - 200 MHz (5.0 ns period)
    synth_clk_period_ns=5.0,
    # Custom build steps
    steps=build_steps,
    # Generate all outputs
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        build_cfg.DataflowOutputType.OOC_SYNTH,
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ],
    # FIFO configuration (like cybersecurity example)
    auto_fifo_depths=True,
    auto_fifo_strategy=build_cfg.AutoFIFOSizingMethod.LARGEFIFO_RTLSIM,
    # Folding configuration
    folding_config_file=None,  # Auto-generate
    # Shell flow for Zynq (required for KV260)
    shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
    # Verbose logging
    verbose=True,
    save_intermediate_models=True,
    # Standalone thresholds (helps with resource usage)
    standalone_thresholds=True,
)

print("✓ Build configuration created (based on FINN cybersecurity example)")
print(f"  Target FPGA: {cfg.fpga_part}")
print(f"  Clock: {1000/cfg.synth_clk_period_ns:.0f} MHz")
print(f"\n  Build steps ({len(cfg.steps)}):")
for i, step_func in enumerate(cfg.steps, 1):
    step_name = step_func.__name__.replace('step_', '').replace('_', ' ').title()
    print(f"    {i:2d}. {step_name}")

print(f"\n  Output types:")
for output_type in cfg.generate_outputs:
    print(f"    - {output_type.name}")

print("\n⚠️  Note: Partitioning step skipped (not required for this model)")
print("   This follows FINN cybersecurity example pattern")

✓ Build configuration created (based on FINN cybersecurity example)
  Target FPGA: xck26-sfvc784-2LV-c
  Clock: 200 MHz

  Build steps (16):
     1. Qonnx To Finn
     2. Tidy Up
     3. Streamline
     4. Convert To Hw
     5. Target Fps Parallelization
     6. Apply Folding Config
     7. Minimize Bit Width
     8. Generate Estimate Reports
     9. Hw Codegen
    10. Hw Ipgen
    11. Set Fifo Depths
    12. Create Stitched Ip
    13. Out Of Context Synthesis
    14. Synthesize Bitfile
    15. Make Pynq Driver
    16. Deployment Package

  Output types:
    - ESTIMATE_REPORTS
    - STITCHED_IP
    - RTLSIM_PERFORMANCE
    - OOC_SYNTH
    - BITFILE
    - DEPLOYMENT_PACKAGE

⚠️  Note: Partitioning step skipped (not required for this model)
   This follows FINN cybersecurity example pattern


In [7]:
# Cell 5.1: Initialize Build

import time

print("="*70)
print("STARTING FINN DATAFLOW BUILD FOR KRIA KV260")
print("="*70)
print(f"\nBuild started: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Model: {model_path}")
print(f"Output: {output_dir}")
print("\n⚠️  This process will take 2-6 hours depending on model complexity.\n")

start_time = time.time()

STARTING FINN DATAFLOW BUILD FOR KRIA KV260

Build started: 2025-12-18 15:16:58
Model: ../../finn_build/ellipse_qonnx_streamlined.qonnx
Output: ../build

⚠️  This process will take 2-6 hours depending on model complexity.



In [8]:
# Cell 5.2: Test FINN Build Import

try:
    import finn.builder.build_dataflow as build
    print("✓ Successfully imported finn.builder.build_dataflow")
    
    # Check if build_dataflow_cfg function exists
    if hasattr(build, 'build_dataflow_cfg'):
        print("✓ build_dataflow_cfg function found")
    else:
        print("✗ build_dataflow_cfg function not found")
        print(f"Available functions: {dir(build)}")
        
except Exception as e:
    print(f"✗ Error importing FINN build: {e}")
    import traceback
    traceback.print_exc()

✓ Successfully imported finn.builder.build_dataflow
✓ build_dataflow_cfg function found


In [9]:
# Cell 5.3: Verify Model and Config

print("Verifying inputs before build...")
print(f"\n1. Model path: {model_path}")
print(f"   Exists: {os.path.exists(model_path)}")

print(f"\n2. Output directory: {output_dir}")
print(f"   Exists: {os.path.exists(output_dir)}")

print(f"\n3. Config object: {type(cfg)}")
print(f"   FPGA part: {cfg.fpga_part}")
print(f"   Clock period: {cfg.synth_clk_period_ns} ns")
print(f"   Number of steps: {len(cfg.steps)}")
print(f"   Output types: {len(cfg.generate_outputs)}")

print("\n✓ Pre-build verification complete")

Verifying inputs before build...

1. Model path: ../../finn_build/ellipse_qonnx_streamlined.qonnx
   Exists: True

2. Output directory: ../build
   Exists: True

3. Config object: <class 'finn.builder.build_dataflow_config.DataflowBuildConfig'>
   FPGA part: xck26-sfvc784-2LV-c
   Clock period: 5.0 ns
   Number of steps: 16
   Output types: 6

✓ Pre-build verification complete


In [10]:
# Cell 5.4.1: Diagnose Conv Nodes

from qonnx.core.modelwrapper import ModelWrapper

print("="*70)
print("DIAGNOSING CONV NODES IN MODEL")
print("="*70)

model = ModelWrapper(model_path)

conv_nodes = [n for n in model.graph.node if n.op_type == 'Conv']
print(f"\nFound {len(conv_nodes)} Conv node(s)")

for i, node in enumerate(conv_nodes, 1):
    print(f"\n{i}. Conv node: {node.name}")
    print(f"   Input: {node.input}")
    print(f"   Output: {node.output}")
    print(f"   Attributes:")
    
    for attr in node.attribute:
        print(f"     - {attr.name}: {attr}")
    
    # Check for missing required attributes
    required_attrs = ['kernel_shape', 'strides', 'pads', 'dilations']
    for req_attr in required_attrs:
        has_attr = any(a.name == req_attr for a in node.attribute)
        status = "✓" if has_attr else "✗ MISSING"
        print(f"   {status} {req_attr}")
    
    # Check weight tensor
    if len(node.input) > 1:
        weight_name = node.input[1]
        if model.get_initializer(weight_name) is not None:
            weight = model.get_initializer(weight_name)
            print(f"   Weight shape: {weight.shape}")
        else:
            print(f"   ⚠ Weight not found in initializers")

print("\n" + "="*70)

DIAGNOSING CONV NODES IN MODEL

Found 4 Conv node(s)

1. Conv node: Conv_0
   Input: ['global_in', 'Conv_0_param0', 'Conv_0_param1']
   Output: ['Conv_0_out0']
   Attributes:
     - auto_pad: name: "auto_pad"
s: "NOTSET"
type: STRING

     - dilations: name: "dilations"
ints: 1
ints: 1
type: INTS

     - group: name: "group"
i: 1
type: INT

     - pads: name: "pads"
ints: 1
ints: 1
ints: 1
ints: 1
type: INTS

     - strides: name: "strides"
ints: 1
ints: 1
type: INTS

   ✗ MISSING kernel_shape
   ✓ strides
   ✓ pads
   ✓ dilations
   Weight shape: (32, 1, 3, 3)

2. Conv node: Conv_1
   Input: ['MaxPool_0_out0', 'Conv_1_param0']
   Output: ['Conv_1_out0']
   Attributes:
     - auto_pad: name: "auto_pad"
s: "NOTSET"
type: STRING

     - dilations: name: "dilations"
ints: 1
ints: 1
type: INTS

     - group: name: "group"
i: 1
type: INT

     - pads: name: "pads"
ints: 1
ints: 1
ints: 1
ints: 1
type: INTS

     - strides: name: "strides"
ints: 1
ints: 1
type: INTS

   ✗ MISSING kernel_shap

In [11]:
# Cell 5.4.2: Fix Conv Nodes (Add Missing Attributes)

from qonnx.core.modelwrapper import ModelWrapper
from onnx import helper
import numpy as np

print("="*70)
print("FIXING CONV NODES")
print("="*70)

model = ModelWrapper(model_path)
modified = False

for node in model.graph.node:
    if node.op_type != 'Conv':
        continue
    
    print(f"\nProcessing Conv node: {node.name}")
    
    # Helper to check if attribute exists
    def has_attr(name):
        return any(a.name == name for a in node.attribute)
    
    # Helper to get attribute value
    def get_attr(name):
        for a in node.attribute:
            if a.name == name:
                if a.ints:
                    return list(a.ints)
                elif a.floats:
                    return list(a.floats)
                elif a.i:
                    return a.i
        return None
    
    # Infer kernel_shape from weight tensor if missing
    if not has_attr('kernel_shape'):
        if len(node.input) > 1:
            weight_name = node.input[1]
            weight = model.get_initializer(weight_name)
            if weight is not None:
                # Weight shape: (M, C, kH, kW) for 2D conv
                k_shape = list(weight.shape[2:4])
                node.attribute.append(helper.make_attribute("kernel_shape", k_shape))
                print(f"  ✓ Added kernel_shape: {k_shape}")
                modified = True
            else:
                print(f"  ✗ Cannot infer kernel_shape - weight not found")
        else:
            print(f"  ✗ Cannot infer kernel_shape - no weight input")
    
    # Add default strides if missing
    if not has_attr('strides'):
        strides = [1, 1]
        node.attribute.append(helper.make_attribute("strides", strides))
        print(f"  ✓ Added strides: {strides}")
        modified = True
    
    # Add default pads if missing
    if not has_attr('pads'):
        pads = [0, 0, 0, 0]
        node.attribute.append(helper.make_attribute("pads", pads))
        print(f"  ✓ Added pads: {pads}")
        modified = True
    
    # Add default dilations if missing
    if not has_attr('dilations'):
        dilations = [1, 1]
        node.attribute.append(helper.make_attribute("dilations", dilations))
        print(f"  ✓ Added dilations: {dilations}")
        modified = True
    
    # Add default group if missing
    if not has_attr('group'):
        node.attribute.append(helper.make_attribute("group", 1))
        print(f"  ✓ Added group: 1")
        modified = True

if modified:
    # Save fixed model
    fixed_model_path = model_path.replace('.qonnx', '_fixed.qonnx')
    model.save(fixed_model_path)
    print(f"\n✓ Fixed model saved to: {fixed_model_path}")
    print(f"\nUpdate model_path to use the fixed model:")
    print(f"  model_path = '{fixed_model_path}'")
else:
    print(f"\n✓ No modifications needed")

print("\n" + "="*70)

FIXING CONV NODES

Processing Conv node: Conv_0
  ✓ Added kernel_shape: [3, 3]

Processing Conv node: Conv_1
  ✓ Added kernel_shape: [3, 3]

Processing Conv node: Conv_2
  ✓ Added kernel_shape: [3, 3]

Processing Conv node: Conv_3
  ✓ Added kernel_shape: [3, 3]

✓ Fixed model saved to: ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx

Update model_path to use the fixed model:
  model_path = '../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx'



In [12]:
# Inspect all intermediate models and their node types
from qonnx.core.modelwrapper import ModelWrapper
from collections import Counter
import glob

intermediate_dir = "../build/intermediate_models/"
intermediate_models = sorted(glob.glob(f"{intermediate_dir}/*.onnx"))
print(f"Found {len(intermediate_models)} intermediate models in {intermediate_dir}")

for model_file in intermediate_models:
    model = ModelWrapper(model_file)
    node_types = Counter([n.op_type for n in model.graph.node])
    print(f"\nModel: {model_file}")
    print(f"  Total nodes: {len(model.graph.node)}")
    print(f"  Node type distribution:")
    for op_type, count in sorted(node_types.items(), key=lambda x: -x[1]):
        print(f"    {op_type}: {count}")

Found 0 intermediate models in ../build/intermediate_models/


In [13]:
# Cell 5.4.3: Update Model Path and Retry Build

# If the previous cell created a fixed model, update the path
import os

fixed_model_path = model_path.replace('.qonnx', '_fixed.qonnx')

if os.path.exists(fixed_model_path):
    print(f"Using fixed model: {fixed_model_path}")
    model_path = fixed_model_path
else:
    print(f"Using original model: {model_path}")

print(f"\nModel path: {model_path}")

Using fixed model: ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx

Model path: ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx


In [14]:
# Cell 5.4.4: Retry Test Build with Fixed Model

print("\n" + "-"*70)
print("RETRYING BUILD WITH FIXED MODEL")
print("-"*70 + "\n")

try:
    from finn.builder.build_dataflow_config import DataflowBuildConfig
    
    test_cfg = DataflowBuildConfig(
        output_dir=output_dir,
        fpga_part=cfg.fpga_part,
        synth_clk_period_ns=cfg.synth_clk_period_ns,
        steps=cfg.steps[:3],
        generate_outputs=[],
        verbose=True,
        save_intermediate_models=True,
    )
    
    print("Starting build with fixed model...")
    build.build_dataflow_cfg(model_path, test_cfg)
    
    print("\n✓ Build successful with fixed model!")
    
except Exception as e:
    print(f"\n✗ Build still failing")
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()


----------------------------------------------------------------------
RETRYING BUILD WITH FIXED MODEL
----------------------------------------------------------------------

Starting build with fixed model...
Building dataflow accelerator from ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx
Intermediate outputs will be generated in /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/build
Final outputs will be generated in ../build
Build log is at ../build/build_dataflow.log
Running step: step_qonnx_to_finn [1/3]
Running step: step_tidy_up [2/3]
Running step: step_streamline [3/3]
Completed successfully

✓ Build successful with fixed model!


/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [15]:
# Inspect all intermediate models and their node types
from qonnx.core.modelwrapper import ModelWrapper
from collections import Counter
import glob

intermediate_dir = "../build/intermediate_models/"
intermediate_models = sorted(glob.glob(f"{intermediate_dir}/*.onnx"))
print(f"Found {len(intermediate_models)} intermediate models in {intermediate_dir}")

for model_file in intermediate_models:
    model = ModelWrapper(model_file)
    node_types = Counter([n.op_type for n in model.graph.node])
    print(f"\nModel: {model_file}")
    print(f"  Total nodes: {len(model.graph.node)}")
    print(f"  Node type distribution:")
    for op_type, count in sorted(node_types.items(), key=lambda x: -x[1]):
        print(f"    {op_type}: {count}")

Found 3 intermediate models in ../build/intermediate_models/

Model: ../build/intermediate_models/step_qonnx_to_finn.onnx
  Total nodes: 63
  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Conv: 4
    MaxPool: 4
    Add: 3
    Gemm: 3
    Reshape: 1

Model: ../build/intermediate_models/step_streamline.onnx
  Total nodes: 76
  Node type distribution:
    Mul: 12
    Where: 12
    Transpose: 8
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Im2Col: 4
    MatMul: 4
    Add: 4
    MaxPoolNHWC: 3
    Gemm: 3
    MaxPool: 1
    Reshape: 1

Model: ../build/intermediate_models/step_tidy_up.onnx
  Total nodes: 63
  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Conv: 4
    MaxPool: 4
    Add: 3
    Gemm: 3
    Reshape: 1


In [16]:
# Inspect step_convert_to_hw.onnx if it exists
from qonnx.core.modelwrapper import ModelWrapper
from collections import Counter
import os

model_file = "../build/intermediate_models/step_convert_to_hw.onnx"
if os.path.exists(model_file):
    model = ModelWrapper(model_file)
    node_types = Counter([n.op_type for n in model.graph.node])
    print(f"Model: {model_file}")
    print(f"  Total nodes: {len(model.graph.node)}")
    print(f"  Node type distribution:")
    for op_type, count in sorted(node_types.items(), key=lambda x: -x[1]):
        print(f"    {op_type}: {count}")
else:
    print(f"{model_file} not found.")

../build/intermediate_models/step_convert_to_hw.onnx not found.


In [17]:
# Cell 5.4.5: Run step_convert_to_hw
from finn.builder.build_dataflow_steps import step_convert_to_hw
from finn.builder.build_dataflow_config import DataflowBuildConfig

print("\n" + "="*70)
print("RUNNING: step_convert_to_hw")
print("="*70 + "\n")

try:
    convert_cfg = DataflowBuildConfig(
        output_dir=output_dir,
        fpga_part=cfg.fpga_part,
        synth_clk_period_ns=cfg.synth_clk_period_ns,
        steps=[step_convert_to_hw],
        generate_outputs=[],
        verbose=True,
        save_intermediate_models=True,
    )
    build.build_dataflow_cfg(model_path, convert_cfg)
    print("\n✓ step_convert_to_hw completed!")
except Exception as e:
    print(f"\n✗ step_convert_to_hw failed: {e}")
    import traceback
    traceback.print_exc()


RUNNING: step_convert_to_hw

Building dataflow accelerator from ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx
Intermediate outputs will be generated in /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/build
Final outputs will be generated in ../build
Build log is at ../build/build_dataflow.log
Running step: step_convert_to_hw [1/1]
Completed successfully

✓ step_convert_to_hw completed!


In [18]:
# Inspect all intermediate models and their node types
from qonnx.core.modelwrapper import ModelWrapper
from collections import Counter
import glob

intermediate_dir = "../build/intermediate_models/"
intermediate_models = sorted(glob.glob(f"{intermediate_dir}/*.onnx"))
print(f"Found {len(intermediate_models)} intermediate models in {intermediate_dir}")

for model_file in intermediate_models:
    model = ModelWrapper(model_file)
    node_types = Counter([n.op_type for n in model.graph.node])
    print(f"\nModel: {model_file}")
    print(f"  Total nodes: {len(model.graph.node)}")
    print(f"  Node type distribution:")
    for op_type, count in sorted(node_types.items(), key=lambda x: -x[1]):
        print(f"    {op_type}: {count}")

Found 4 intermediate models in ../build/intermediate_models/

Model: ../build/intermediate_models/step_convert_to_hw.onnx
  Total nodes: 63
  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Conv: 4
    MaxPool: 4
    Add: 3
    Gemm: 3
    Reshape: 1

Model: ../build/intermediate_models/step_qonnx_to_finn.onnx
  Total nodes: 63
  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Conv: 4
    MaxPool: 4
    Add: 3
    Gemm: 3
    Reshape: 1

Model: ../build/intermediate_models/step_streamline.onnx
  Total nodes: 76
  Node type distribution:
    Mul: 12
    Where: 12
    Transpose: 8
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Im2Col: 4
    MatMul: 4
    Add: 4
    MaxPoolNHWC: 3
    Gemm: 3
    MaxPool: 1
    Reshape: 1

Model: ../build/intermediate_models/step_tidy_up.onnx
  Total nodes: 63
  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round

In [19]:
# Cell 5.4.6: Inspect step_convert_to_hw.onnx
from qonnx.core.modelwrapper import ModelWrapper
from collections import Counter
import os

model_file = "../build/intermediate_models/step_convert_to_hw.onnx"
if os.path.exists(model_file):
    model = ModelWrapper(model_file)
    node_types = Counter([n.op_type for n in model.graph.node])
    print(f"Model: {model_file}")
    print(f"  Total nodes: {len(model.graph.node)}")
    print(f"  Node type distribution:")
    for op_type, count in sorted(node_types.items(), key=lambda x: -x[1]):
        print(f"    {op_type}: {count}")
else:
    print(f"{model_file} not found.")

Model: ../build/intermediate_models/step_convert_to_hw.onnx
  Total nodes: 63
  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Conv: 4
    MaxPool: 4
    Add: 3
    Gemm: 3
    Reshape: 1


In [20]:
# Inspect intermediate model before estimate reports
from qonnx.core.modelwrapper import ModelWrapper
from collections import Counter
import glob

interm_dir = "../build/intermediate_models/"
interm_models = sorted(glob.glob(f"{interm_dir}/*.onnx"))
print(f"Intermediate models: {interm_models[-3:]}")  # Show last 3

# Pick the latest model before the error
if interm_models:
    model = ModelWrapper(interm_models[-1])
    node_types = Counter([n.op_type for n in model.graph.node])
    print(f"Model: {interm_models[-1]}")
    print(f"  Total nodes: {len(model.graph.node)}")
    print(f"  Node type distribution:")
    for op_type, count in sorted(node_types.items(), key=lambda x: -x[1]):
        print(f"    {op_type}: {count}")
else:
    print("No intermediate models found.")

Intermediate models: ['../build/intermediate_models/step_qonnx_to_finn.onnx', '../build/intermediate_models/step_streamline.onnx', '../build/intermediate_models/step_tidy_up.onnx']
Model: ../build/intermediate_models/step_tidy_up.onnx
  Total nodes: 63
  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Conv: 4
    MaxPool: 4
    Add: 3
    Gemm: 3
    Reshape: 1


In [21]:
# Cell 5.4.7: Check for FINN hardware nodes in step_convert_to_hw.onnx
from qonnx.core.modelwrapper import ModelWrapper

model_file = "../build/intermediate_models/step_convert_to_hw.onnx"
if os.path.exists(model_file):
    model = ModelWrapper(model_file)
    op_types = set([n.op_type for n in model.graph.node])
    print("Unique op types in step_convert_to_hw.onnx:", op_types)
    # List common FINN hardware node types
    finn_hw_ops = [
        "StreamingFCLayer_Batch", "StreamingMaxPool_Batch", "StreamingConv", 
        "StreamingDataflowPartition", "Thresholding_Batch", "MultiThreshold", "TLastMarker"
    ]
    found_hw = [op for op in op_types if op in finn_hw_ops]
    if found_hw:
        print("✓ FINN hardware nodes found:", found_hw)
    else:
        print("⚠️  No FINN hardware nodes found. Your model may not be quantized or is not compatible with FINN hardware conversion.")
        print("    Please ensure your model is quantized and uses only FINN-supported layers.")
else:
    print(f"{model_file} not found.")

Unique op types in step_convert_to_hw.onnx: {'MaxPool', 'Conv', 'Mul', 'Greater', 'Round', 'Reshape', 'Gemm', 'Add', 'Less', 'Where', 'Relu'}
⚠️  No FINN hardware nodes found. Your model may not be quantized or is not compatible with FINN hardware conversion.
    Please ensure your model is quantized and uses only FINN-supported layers.


In [22]:
# Cell 5.4.8: Diagnose why hardware nodes are missing

print("="*70)
print("DIAGNOSIS: No FINN hardware nodes found after step_convert_to_hw")
print("="*70)
print("""
Possible reasons:
  1. The model is not quantized (weights/activations are floating-point).
  2. The model contains unsupported ONNX ops for FINN hardware.
  3. The export from Brevitas or quantization step was skipped or failed.

Recommended actions:
  - Ensure you use Brevitas quantized layers (QuantConv2d, QuantLinear, etc.).
  - Export your model to QONNX using Brevitas export.
  - After export, check that all weights and activations are quantized (int2, int4, int8, etc.).
  - The model should contain only FINN-supported ops (see: https://finn.readthedocs.io/en/latest/onnx_supported_ops.html).

To check quantization, you can inspect the data types of weights in your model:
""")

from qonnx.core.modelwrapper import ModelWrapper
import numpy as np

model_file = "../build/intermediate_models/step_tidy_up.onnx"
model = ModelWrapper(model_file)
for n in model.graph.node:
    if n.op_type in ["Conv", "Gemm"]:
        w_name = n.input[1]
        w_tensor = model.get_initializer(w_name)
        if w_tensor is not None:
            print(f"{n.op_type} node '{n.name}': weights dtype = {w_tensor.dtype}, shape = {w_tensor.shape}")
        else:
            print(f"{n.op_type} node '{n.name}': weights not found in initializers")

print("\nIf you see 'float32' weights, your model is NOT quantized. You must retrain/export with quantization.")
print("="*70)

DIAGNOSIS: No FINN hardware nodes found after step_convert_to_hw

Possible reasons:
  1. The model is not quantized (weights/activations are floating-point).
  2. The model contains unsupported ONNX ops for FINN hardware.
  3. The export from Brevitas or quantization step was skipped or failed.

Recommended actions:
  - Ensure you use Brevitas quantized layers (QuantConv2d, QuantLinear, etc.).
  - Export your model to QONNX using Brevitas export.
  - After export, check that all weights and activations are quantized (int2, int4, int8, etc.).
  - The model should contain only FINN-supported ops (see: https://finn.readthedocs.io/en/latest/onnx_supported_ops.html).

To check quantization, you can inspect the data types of weights in your model:

Conv node 'Conv_0': weights dtype = float32, shape = (32, 1, 3, 3)
Conv node 'Conv_1': weights dtype = float32, shape = (64, 32, 3, 3)
Conv node 'Conv_2': weights dtype = float32, shape = (128, 64, 3, 3)
Conv node 'Conv_3': weights dtype = float32

In [21]:
# Cell: Attempt to partition and convert to hardware nodes
from qonnx.transformation.create_dataflow_partition import CreateDataflowPartition
from finn.transformation.fpgadataflow.convert_to_hls_layers import ConvertToHLSLayers
from qonnx.core.modelwrapper import ModelWrapper

# Load the latest streamlined or tidy model
hw_model_path = "../build/intermediate_models/step_tidy_up.onnx"
model = ModelWrapper(hw_model_path)

# Try to partition the model for dataflow
model = model.transform(CreateDataflowPartition())

# Try to convert to HLS layers (hardware nodes)
model = model.transform(ConvertToHLSLayers())

# Save the hardware-mapped model
hw_out_path = "../build/intermediate_models/manual_hw_partition.onnx"
model.save(hw_out_path)
print(f"✓ Hardware-mapped model saved to: {hw_out_path}")

# Inspect node types
from collections import Counter
node_types = Counter([n.op_type for n in model.graph.node])
print(f"  Total nodes: {len(model.graph.node)}")
print(f"  Node type distribution:")
for op_type, count in sorted(node_types.items(), key=lambda x: -x[1]):
    print(f"    {op_type}: {count}")

ModuleNotFoundError: No module named 'qonnx.transformation.create_dataflow_partition'

In [22]:
# List all unique op types in your model
from qonnx.core.modelwrapper import ModelWrapper

model = ModelWrapper("../build/intermediate_models/step_tidy_up.onnx")
op_types = set([n.op_type for n in model.graph.node])
print("Unique op types:", op_types)

Unique op types: {'Greater', 'Mul', 'Conv', 'Where', 'Less', 'MaxPool', 'Add', 'Reshape', 'Relu', 'Gemm', 'Round'}


In [ ]:
# Cell 5.4.7: Run remaining build steps (hardware generation to deployment)
from finn.builder.build_dataflow_steps import (
    step_target_fps_parallelization,
    step_apply_folding_config,
    step_minimize_bit_width,
    step_generate_estimate_reports,
    step_hw_codegen,
    step_hw_ipgen,
    step_set_fifo_depths,
    step_create_stitched_ip,
    step_out_of_context_synthesis,
    step_synthesize_bitfile,
    step_make_pynq_driver,
    step_deployment_package,
)
from finn.builder.build_dataflow_config import DataflowBuildConfig

print("\n" + "="*70)
print("RUNNING: Remaining FINN build steps (hardware generation to deployment)")
print("="*70 + "\n")

remaining_steps = [
    step_target_fps_parallelization,
    step_apply_folding_config,
    step_minimize_bit_width,
    step_generate_estimate_reports,
    step_hw_codegen,
    step_hw_ipgen,
    step_set_fifo_depths,
    step_create_stitched_ip,
    step_out_of_context_synthesis,
    step_synthesize_bitfile,
    step_make_pynq_driver,
    step_deployment_package,
]

try:
    hw_cfg = DataflowBuildConfig(
        output_dir=output_dir,
        fpga_part=cfg.fpga_part,
        synth_clk_period_ns=cfg.synth_clk_period_ns,
        steps=remaining_steps,
        generate_outputs=[
            build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
            build_cfg.DataflowOutputType.STITCHED_IP,
            build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
            build_cfg.DataflowOutputType.OOC_SYNTH,
            build_cfg.DataflowOutputType.BITFILE,
            build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
        ],
        auto_fifo_depths=True,
        auto_fifo_strategy=build_cfg.AutoFIFOSizingMethod.LARGEFIFO_RTLSIM,
        folding_config_file=None,
        shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
        verbose=True,
        save_intermediate_models=True,
        standalone_thresholds=True,
    )
    # Use the output of step_convert_to_hw as input
    convert_model_path = "../build/intermediate_models/step_convert_to_hw.onnx"
    build.build_dataflow_cfg(convert_model_path, hw_cfg)
    print("\n✓ Remaining build steps completed!")
except Exception as e:
    print(f"\n✗ Remaining build steps failed: {e}")
    import traceback
    traceback.print_exc()


RUNNING: Remaining FINN build steps (hardware generation to deployment)

Building dataflow accelerator from ../build/intermediate_models/step_convert_to_hw.onnx
Intermediate outputs will be generated in /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/build
Final outputs will be generated in ../build
Build log is at ../build/build_dataflow.log
Running step: step_target_fps_parallelization [1/12]
Running step: step_apply_folding_config [2/12]
Running step: step_minimize_bit_width [3/12]
Running step: step_generate_estimate_reports [4/12]
> /home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/finn/analysis/fpgadataflow/dataflow_performance.py(73)dataflow_performance()
     71                     max_pred_latency = max(pred_latencies)
     72                 latency_at_node_output[node.name] = node_cycles + max_pred_latency
---> 73     critical_path_cycles = max(latency_at_node_output.values())
     74     return {
     75         "cri

Traceback (most recent call last):
  File "/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/finn/builder/build_dataflow.py", line 158, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/finn/builder/build_dataflow_steps.py", line 487, in step_generate_estimate_reports
    estimate_network_performance = model.analysis(dataflow_performance)
  File "/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/qonnx/core/modelwrapper.py", line 129, in analysis
    return analysis_fxn(self)
  File "/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/finn/analysis/fpgadataflow/dataflow_performance.py", line 73, in dataflow_performance
    critical_path_cycles = max(latency_at_node_output.values())
ValueError: max() arg is an empty sequence


In [ ]:
# --- Replace Cell 5.5.1 with the following to run the full build and generate the bitstream ---

# Cell 5.5.1: Run Build - Generate Bitstream (FULL BUILD)

print("\n" + "="*70)
print("RUNNING FULL BUILD - BITSTREAM GENERATION")
print("="*70 + "\n")

try:
    from finn.builder.build_dataflow_config import DataflowBuildConfig
    from finn.builder.build_dataflow_steps import (
        step_qonnx_to_finn,
        step_tidy_up,
        step_streamline,
        step_convert_to_hw,
        step_target_fps_parallelization,
        step_apply_folding_config,
        step_minimize_bit_width,
        step_generate_estimate_reports,
        step_hw_codegen,
        step_hw_ipgen,
        step_set_fifo_depths,
        step_create_stitched_ip,
        step_out_of_context_synthesis,
        step_synthesize_bitfile,
        step_make_pynq_driver,
        step_deployment_package,
    )

    # Configuration - FULL build (all steps, including bitstream)
    full_cfg = DataflowBuildConfig(
        output_dir=output_dir,
        fpga_part="xck26-sfvc784-2LV-c",
        synth_clk_period_ns=5.0,
        steps=[
            step_qonnx_to_finn,           # 1
            step_tidy_up,                 # 2
            step_streamline,              # 3
            step_convert_to_hw,           # 4
            step_target_fps_parallelization,  # 5
            step_apply_folding_config,    # 6
            step_minimize_bit_width,      # 7
            step_generate_estimate_reports, # 8
            step_hw_codegen,              # 9
            step_hw_ipgen,                # 10
            step_set_fifo_depths,         # 11
            step_create_stitched_ip,      # 12
            step_out_of_context_synthesis,# 13
            step_synthesize_bitfile,      # 14
            step_make_pynq_driver,        # 15
            step_deployment_package,      # 16
        ],
        generate_outputs=[
            build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
            build_cfg.DataflowOutputType.STITCHED_IP,
            build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
            build_cfg.DataflowOutputType.OOC_SYNTH,
            build_cfg.DataflowOutputType.BITFILE,
            build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
        ],
        auto_fifo_depths=True,
        auto_fifo_strategy=build_cfg.AutoFIFOSizingMethod.LARGEFIFO_RTLSIM,
        folding_config_file=None,
        shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
        verbose=True,
        save_intermediate_models=True,
        standalone_thresholds=True,
    )

    print("Starting full build (bitstream generation)...")
    print("  ✓ All steps will be run, including bitstream synthesis")
    print("  ⏱️  Estimated time: 2-6 hours\n")

    import time
    start_time = time.time()

    build.build_dataflow_cfg(model_path, full_cfg)

    elapsed = (time.time() - start_time) / 60

    print("\n" + "="*70)
    print("✓ FULL BUILD COMPLETED SUCCESSFULLY")
    print("="*70)
    print(f"\n  Build time: {elapsed:.1f} minutes")
    print(f"\n  Bitstream and deployment package generated in:")
    print(f"    {output_dir}/")
    print(f"\n  Next steps:")
    print(f"    1. Check the build results and deployment package.")
    print(f"    2. Copy the files to your KV260 board.")
    print(f"    3. Use the provided driver and test script for inference.")
    print("")

except Exception as e:
    print(f"\n✗ Build failed: {e}")
    import traceback
    traceback.print_exc()


RUNNING FULL BUILD - BITSTREAM GENERATION

Starting full build (bitstream generation)...
  ✓ All steps will be run, including bitstream synthesis
  ⏱️  Estimated time: 2-6 hours

Building dataflow accelerator from ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx
Intermediate outputs will be generated in /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/build
Final outputs will be generated in ../build
Build log is at ../build/build_dataflow.log
Running step: step_qonnx_to_finn [1/16]
Running step: step_tidy_up [2/16]
Running step: step_streamline [3/16]
Running step: step_convert_to_hw [4/16]
Running step: step_target_fps_parallelization [5/16]
Running step: step_apply_folding_config [6/16]
Running step: step_minimize_bit_width [7/16]


/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/finn/transformation/fpgadataflow/convert_to_hw_layers.py:63: UserWarning: Im2Col_0 : Input is not int. Can't infer ConvInpGen.
  warnings.warn("%s : Input is not int. Can't infer ConvInpGen." % n.name)
/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/finn/transformation/fpgadataflow/convert_to_hw_layers.py:63: UserWarning: Im2Col_1 : Input is not int. Can't infer ConvInpGen.
  warnings.warn("%s : Input is not int. Can't infer ConvInpGen." % n.name)
/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/finn/transformation/fpgadataflow/convert_to_hw_layers.py:63: UserWarning: Im2Col_2 : Input is not int. Can't infer ConvInpGen.
  warnings.warn("%s : Input is not int. Can't infer ConvInpGen." % n.name)
/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/finn/transformation/fpgadataflow/convert_to_hw_layers.py:63: UserWarning: Im2Col_3 : Input is not int. Can't infer

Running step: step_generate_estimate_reports [8/16]
> /home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/finn/analysis/fpgadataflow/dataflow_performance.py(73)dataflow_performance()
     71                     max_pred_latency = max(pred_latencies)
     72                 latency_at_node_output[node.name] = node_cycles + max_pred_latency
---> 73     critical_path_cycles = max(latency_at_node_output.values())
     74     return {
     75         "critical_path_cycles": int(critical_path_cycles),



In [14]:
# Cell 5.5.2: Check Generated IP Cores (After Step 9)

import os
from pathlib import Path

print("="*70)
print("CHECKING GENERATED IP CORES FROM STEP 9")
print("="*70)

build_path = Path(output_dir)

# Look for intermediate models directory
intermediate_dir = build_path / "intermediate_models"
if intermediate_dir.exists():
    print(f"\n✓ Found intermediate_models directory")
    
    # Find all IP-related directories
    ip_dirs = []
    for item in intermediate_dir.rglob("*"):
        if item.is_dir() and ("ipgen" in item.name or item.name == "ip"):
            ip_dirs.append(item)
    
    if ip_dirs:
        print(f"\n✓ Found {len(ip_dirs)} IP generation directories:")
        for ip_dir in sorted(ip_dirs):
            rel_path = ip_dir.relative_to(build_path)
            print(f"\n  {rel_path}/")
            
            # List IP files
            ip_files = list(ip_dir.glob("*.zip")) + list(ip_dir.glob("*.tcl")) + list(ip_dir.glob("component.xml"))
            if ip_files:
                print(f"    Contains {len(ip_files)} IP files:")
                for f in ip_files[:5]:
                    print(f"      - {f.name}")
                if len(ip_files) > 5:
                    print(f"      ... and {len(ip_files)-5} more")
    else:
        print("\n✗ No IP directories found")
else:
    print(f"\n✗ intermediate_models directory not found")

# Also check for any .zip IP packages
print("\n" + "-"*70)
print("SEARCHING FOR IP PACKAGES (.zip)")
print("-"*70)

zip_files = list(build_path.rglob("*.zip"))
if zip_files:
    print(f"\n✓ Found {len(zip_files)} .zip files:")
    for zf in zip_files[:10]:
        rel_path = zf.relative_to(build_path)
        size_kb = zf.stat().st_size / 1024
        print(f"  {rel_path} ({size_kb:.1f} KB)")
    if len(zip_files) > 10:
        print(f"  ... and {len(zip_files)-10} more")
else:
    print("\n⚠️  No .zip IP packages found")

print("\n" + "="*70)

CHECKING GENERATED IP CORES FROM STEP 9

✓ Found intermediate_models directory

✗ No IP directories found

----------------------------------------------------------------------
SEARCHING FOR IP PACKAGES (.zip)
----------------------------------------------------------------------

⚠️  No .zip IP packages found



In [15]:
# Cell 6: Analyze Build Results

import os
import json
from pathlib import Path

print("="*70)
print("BUILD RESULTS ANALYSIS")
print("="*70)

if not os.path.exists(output_dir):
    print("✗ Build directory not found!")
else:
    print(f"\n✓ Build directory: {output_dir}\n")
    
    # Find key files
    build_path = Path(output_dir)
    
    # 1. Bitstream
    bitfiles = list(build_path.rglob("*.bit"))
    if bitfiles:
        print(f"✓ Bitstream Files ({len(bitfiles)}):")
        for bf in bitfiles:
            size_mb = bf.stat().st_size / (1024*1024)
            print(f"    {bf.relative_to(build_path)} ({size_mb:.2f} MB)")
    else:
        print("⚠ No bitstream (.bit) files found")
    
    # 2. Hardware handoff
    hwh_files = list(build_path.rglob("*.hwh"))
    if hwh_files:
        print(f"\n✓ Hardware Handoff Files ({len(hwh_files)}):")
        for hwh in hwh_files:
            print(f"    {hwh.relative_to(build_path)}")
    
    # 3. Driver
    driver_files = list(build_path.rglob("driver.py"))
    if driver_files:
        print(f"\n✓ Driver Files ({len(driver_files)}):")
        for df in driver_files:
            print(f"    {df.relative_to(build_path)}")
    
    # 4. Reports
    report_files = list(build_path.rglob("*estimate*.json"))
    if report_files:
        print(f"\n✓ Resource Estimates ({len(report_files)}):")
        for rf in sorted(report_files)[:3]:
            print(f"    {rf.relative_to(build_path)}")
            try:
                with open(rf) as f:
                    data = json.load(f)
                    if 'total' in data:
                        print(f"      LUT: {data['total'].get('LUT', 'N/A')}")
                        print(f"      FF: {data['total'].get('FF', 'N/A')}")
                        print(f"      BRAM: {data['total'].get('BRAM_18K', 'N/A')}")
                        print(f"      DSP: {data['total'].get('DSP', 'N/A')}")
            except:
                pass
    
    # 5. Deployment package
    deploy_dirs = [d for d in build_path.rglob("deploy*") if d.is_dir()]
    if deploy_dirs:
        print(f"\n✓ Deployment Package:")
        for dd in deploy_dirs:
            print(f"    {dd.relative_to(build_path)}/")
            deploy_files = list(dd.iterdir())[:5]
            for df in deploy_files:
                print(f"      - {df.name}")
    
    # 6. Intermediate models
    intermediate_models = list(build_path.rglob("intermediate_models/*.onnx"))
    if intermediate_models:
        print(f"\n✓ Intermediate Models ({len(intermediate_models)}):")
        for im in sorted(intermediate_models)[:5]:
            print(f"    {im.name}")
        if len(intermediate_models) > 5:
            print(f"    ... and {len(intermediate_models) - 5} more")
    
    print("\n" + "="*70)

BUILD RESULTS ANALYSIS

✓ Build directory: ../build

⚠ No bitstream (.bit) files found

✓ Resource Estimates (1):
    report/estimate_layer_resources_hls.json

✓ Intermediate Models (9):
    step_apply_folding_config.onnx
    step_convert_to_hw.onnx
    step_hw_codegen.onnx
    step_hw_ipgen.onnx
    step_minimize_bit_width.onnx
    ... and 4 more



In [16]:
# Cell 7: Create Deployment Package for KV260

import shutil
from pathlib import Path
import time

print("="*70)
print("CREATING DEPLOYMENT PACKAGE FOR KV260")
print("="*70)

# Create deployment directory
deploy_output = Path(output_dir) / "kv260_deployment"
deploy_output.mkdir(exist_ok=True)

print(f"\n✓ Deployment directory: {deploy_output}\n")

build_path = Path(output_dir)
files_copied = 0

# Copy bitstream
bitfiles = list(build_path.rglob("*.bit"))
if bitfiles:
    shutil.copy(bitfiles[0], deploy_output / "finn_accel.bit")
    print(f"  ✓ Copied: finn_accel.bit")
    files_copied += 1
else:
    print(f"  ⚠ No bitstream found")

# Copy hardware handoff
hwh_files = list(build_path.rglob("*.hwh"))
if hwh_files:
    shutil.copy(hwh_files[0], deploy_output / "finn_accel.hwh")
    print(f"  ✓ Copied: finn_accel.hwh")
    files_copied += 1
else:
    print(f"  ⚠ No .hwh file found")

# Copy driver
driver_files = list(build_path.rglob("driver.py"))
if driver_files:
    shutil.copy(driver_files[0], deploy_output / "driver.py")
    print(f"  ✓ Copied: driver.py")
    files_copied += 1
else:
    print(f"  ⚠ No driver found")

# Copy reports
report_files = list(build_path.rglob("*estimate*.json"))
if report_files:
    shutil.copy(report_files[0], deploy_output / "resource_estimate.json")
    print(f"  ✓ Copied: resource_estimate.json")
    files_copied += 1

# Create README
readme = f"""# Ellipse Regression FINN Accelerator - Kria KV260 Deployment

## Overview

This package contains the FINN-generated accelerator for ellipse regression on Kria KV260.

## Files

- `finn_accel.bit` - Bitstream for Kria KV260
- `finn_accel.hwh` - Hardware handoff file  
- `driver.py` - Python driver code
- `resource_estimate.json` - Resource utilization report

## Deployment to KV260

### 1. Copy Files to KV260

```bash
scp -r {deploy_output.name} xilinx@<kv260-ip>:~/
```

### 2. On KV260, Load the Overlay

```python
from pynq import Overlay

# Load the accelerator
overlay = Overlay("finn_accel.bit")

# Access the accelerator (adjust based on your driver)
accel = overlay.finn_accel_0
```

### 3. Run Inference

```python
import numpy as np

# Prepare input data
input_data = np.random.randint(0, 255, size=(1, 3, 224, 224), dtype=np.uint8)

# Run inference
output = accel.execute(input_data)

print("Ellipse parameters:", output)
```

## Build Information

- **Target**: Kria KV260 (xck26-sfvc784-2LV-c)
- **Clock**: 200 MHz
- **Build date**: {time.strftime('%Y-%m-%d %H:%M:%S')}
- **FINN version**: Check build logs

## Resource Usage

See `resource_estimate.json` for detailed resource utilization.

## Troubleshooting

1. **Overlay won't load**: Check that bitstream matches FPGA part
2. **Driver import error**: Ensure PYNQ is installed on KV260
3. **Execution hangs**: Check FIFO depths and DMA configuration

## References

- FINN Documentation: https://finn.readthedocs.io
- Kria KV260: https://www.xilinx.com/products/som/kria/kv260-vision-starter-kit.html
- PYNQ: http://www.pynq.io
"""

with open(deploy_output / "README.md", 'w') as f:
    f.write(readme)
print(f"  ✓ Created: README.md")
files_copied += 1

# Create a simple test script
test_script = """#!/usr/bin/env python3
\"\"\"
Simple test script for FINN accelerator on Kria KV260
\"\"\"

import numpy as np
from pynq import Overlay
import time

print("Loading FINN accelerator overlay...")
overlay = Overlay("finn_accel.bit")
print("✓ Overlay loaded")

# Get accelerator handle
# Note: Adjust 'finn_accel_0' to match your actual IP name
try:
    accel = overlay.finn_accel_0
    print("✓ Accelerator found")
except:
    print("✗ Could not find accelerator IP")
    print("Available IPs:", dir(overlay))
    exit(1)

# Prepare test input
print("\\nPreparing test input...")
input_shape = (1, 3, 224, 224)  # Adjust to your model input
input_data = np.random.randint(0, 255, size=input_shape, dtype=np.uint8)
print(f"  Input shape: {input_shape}")

# Run inference
print("\\nRunning inference...")
start = time.time()
output = accel.execute(input_data)
elapsed = time.time() - start

print(f"✓ Inference complete in {elapsed*1000:.2f} ms")
print(f"  Output shape: {output.shape}")
print(f"  Output: {output}")

print("\\n✓ Test completed successfully!")
"""

with open(deploy_output / "test_kv260.py", 'w') as f:
    f.write(test_script)
print(f"  ✓ Created: test_kv260.py")
files_copied += 1

print(f"\n✓ Deployment package complete: {files_copied} files")
print(f"\n📦 Ready to deploy:")
print(f"   scp -r {deploy_output.name} xilinx@<kv260-ip>:~/")

print("\n" + "="*70)

CREATING DEPLOYMENT PACKAGE FOR KV260

✓ Deployment directory: ../build/kv260_deployment

  ⚠ No bitstream found
  ⚠ No .hwh file found
  ⚠ No driver found
  ✓ Copied: resource_estimate.json
  ✓ Created: README.md
  ✓ Created: test_kv260.py

✓ Deployment package complete: 3 files

📦 Ready to deploy:
   scp -r kv260_deployment xilinx@<kv260-ip>:~/

